In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import time 
import spacy

from tqdm import tqdm
from openai import OpenAI

In [2]:
VERSION = 0

In [3]:
df = pd.read_csv('../persuade_2.0_human.csv')
df.head()

,essay_id_comp,full_text,holistic_essay_score,word_count,prompt_name,task,assignment,source_text,gender,grade_level,ell_status,race_ethnicity,economically_disadvantaged,student_disability_status
0,423A1CA112E2,Phones\n\nModern humans today are always on th...,3,378,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
1,BC75783F96E3,This essay will explain if drivers should or s...,4,432,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
2,74C8BC7417DE,Driving while the use of cellular devices\n\nT...,2,179,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,F,NaN,NaN,White,NaN,NaN
3,A8445CABFECE,Phones & Driving\n\nDrivers should not be able...,3,221,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
4,6B4F7A0165B9,Cell Phone Operation While Driving\n\nThe abil...,4,334,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,White,NaN,NaN


In [4]:
df = df[VERSION * 1000:min((VERSION + 1) * 1000, len(df))]

In [5]:
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI()

In [6]:
def generate_text(model_input, idx):
    messages = [
        {
            'role': 'user',
            'content': model_input
        }
    ]

    random_temperature = random.uniform(0.0, 1.0)
    random_top_p = random.uniform(0.9, 1.0)

    flag = False

    for attempt in range(5):
        try:
            response = client.chat.completions.create(
                model='gpt-4-1106-preview',
                messages=messages,
                max_tokens=4096,
                frequency_penalty=1.12,
                temperature=random_temperature,
                top_p=random_top_p
            )
            output = response.choices[0].message.content
            flag = True
            print(f'Generated Successfully on Idx-{idx}!!!')
            return output
            break
        except Exception as e:
            print(f'Attempt {attempt + 1} on Idx-{idx} Failed: {e}')
            time.sleep(1)

    if flag == False:
        print(f'Idx-{idx} Failed at Last')
        return None

In [7]:
file_name = f"piidd_dataset_{VERSION}.csv"

if os.path.exists(file_name):
    print(f"The file {file_name} exists.")
    save_df = pd.read_csv(file_name)
else:
    print(f"The file {file_name} does not exist.")
    save_df = pd.DataFrame(columns=["full_text"])
    save_df.to_csv(f"piidd_dataset_{VERSION}.csv", index=False)

The file piidd_dataset_0.csv exists.


In [8]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    original_text = row['full_text']

    prompt = """
It is known that there are currently seven main types of PII (Personally Identifiable Information):
(1) NAME_STUDENT - The full or partial name of a student that is not necessarily the author of the essay. This excludes instructors, authors, and other person names.
(2) EMAIL - A student's email address.
(3) USERNAME - A student's username on any platform.
(4) ID_NUM - A number or sequence of characters that could be used to identify a student, such as a student ID or a social security number.
(5) PHONE_NUM - A phone number associated with a student.
(6) URL_PERSONAL - A URL that might be used to identify a student.
(7) STREET_ADDRESS - A full or partial street address that is associated with the student, such as their home address.

Now I will give you an article, and you need to complete the following tasks:
(1) Add information related to the aforementioned seven tags into the article. This information must be relevant to individual students while still maintaining the article's coherence, fluidity, readability and core theme.
(2) Replace the newly added relevant information with real data.
(3) You only need to return the revised article; do not include any extraneous content before or after the article.
(4) Enclose the newly added content with tags before and after, for example: [NAME_STUDENT]James Jones[NAME_STUDENT].
(5) Suppose the original text already contains information related to the seven tags; you also need to enclose them with tags as per requirement (4).
(6) The positional distribution of different labels should be average in the whole article.
(7) Do not forcibly add a label, all content must be reasonable.
(8) The article must not contain grammatical errors or misspellings.

Article:
    """

    model_input_for_new_text_generation = prompt + "\n" + original_text
    print(model_input_for_new_text_generation)

    new_text = generate_text(model_input=model_input_for_new_text_generation, idx=idx)
    print(new_text)
    
    new_row = pd.DataFrame({
        "full_text": [new_text]
    })
    save_df = pd.concat([save_df, new_row], ignore_index=True)
    save_df.to_csv(f"piidd_dataset_{VERSION}.csv", index=False)

    break

  0%|          | 0/1000 [00:00<?, ?it/s]


It is known that there are currently seven main types of PII (Personally Identifiable Information):
(1) NAME_STUDENT - The full or partial name of a student that is not necessarily the author of the essay. This excludes instructors, authors, and other person names.
(2) EMAIL - A student's email address.
(3) USERNAME - A student's username on any platform.
(4) ID_NUM - A number or sequence of characters that could be used to identify a student, such as a student ID or a social security number.
(5) PHONE_NUM - A phone number associated with a student.
(6) URL_PERSONAL - A URL that might be used to identify a student.
(7) STREET_ADDRESS - A full or partial street address that is associated with the student, such as their home address.

Now I will give you an article, and you need to complete the following tasks:
(1) Add information related to the aforementioned seven tags into the article. This information must be relevant to individual students while still maintaining the article's cohe

  0%|          | 0/1000 [00:34<?, ?it/s]

Generated Successfully on Idx-0!!!
Phones

Modern humans today are always on their phone. [NAME_STUDENT]Sarah Thompson[NAME_STUDENT] noticed her peers are often engaged with their phones for more than 5 hours a day non-stop. All they do is text back and forth and just have group chats on social media platforms like [USERNAME]CoolKid1992[USERNAME]. They even do it while driving. There are some really bad consequences when things happen in relation to phone usage. Some certain areas in the United States ban phones from classrooms, just because of these issues.

When people have phones, they know about certain apps that they use daily. Apps like Facebook, Twitter, Instagram, and Snapchat help maintain connections despite distance or time zones. For instance, when [EMAIL]markus.journey@email.com[EMAIL]'s friend moved away to [STREET_ADDRESS]742 Evergreen Terrace[STREET_ADDRESS], he was able to stay in contact by posting videos or sending text messages through these platforms.

Driving is o

In [9]:
save_df = pd.read_csv(f"piidd_dataset_{VERSION}.csv")
save_df.head()

,full_text
0,Phones\n\nModern humans today are always on th...
1,Phones\n\nModern humans today are always on th...
